# Demonstrating how significance of EC difference can be tested

In [ ]:
# making sure that updates to imported files are immediately available without restarting the kernel
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from scipy import stats

# not using this here because finer control needed
# font = {"size": 15}
# matplotlib.rc("font", **font)

sys.path.append(os.path.abspath(".."))
from utils import fast_cohen, simulate_trials_from_copy_model

In [ ]:
# load the bootstrapped data
df = pd.read_parquet(
    "data/model_wise_bootstrapped_ecs_standard_1000.parquet", engine="pyarrow"
)
df["model"] = df["model"].astype(str)

# take the mean over conditions
exp_mean_df = df.groupby(
    ["bootstrap_id", "experiment", "model"], observed=True, as_index=False
).mean(numeric_only=True)

# take the average across the experiments
mean_df = exp_mean_df.groupby(
    ["bootstrap_id", "model"], observed=True, as_index=False
).mean(numeric_only=True)

final_df = mean_df.groupby("model")["model-human-ec"].mean().reset_index()
display(final_df)

models = final_df.sort_values(by="model-human-ec", ascending=False)["model"].values
print(models)

## Naive Approach
Logic is: Select two models and humans, bootstrap (I have the data already) and compare the distribution of EC-deltas to the null hypothesis of no difference with a two-sided t-test.

In [ ]:
def calc_p_value(df, m1, m2, plot=True):
    """
    This calculates the p-value by doing a 1-sample t-test between the distribution of EC deltas and population mean 0.
    The problem with this is that we get very small p-values simply because of doing so many bootstraps, but what this is neglecting is the noise
    incurred by different participants etc (i.e. our sample is fixed).

    I'm still keeping the code for future reference, and because the plotting is still useful.

    :param m1: name of model 1
    :param m2: name of model 2
    """

    df = df[(df["model"].isin([m1, m2]))]

    # printing the CIs
    lower, upper = np.percentile(
        mean_df[mean_df["model"] == m1]["model-human-ec"].to_numpy(), [2.5, 97.5]
    )
    print(f"95% interval for {m1}: [{lower:.3f}, {upper:.3f}]")
    lower, upper = np.percentile(
        mean_df[mean_df["model"] == m2]["model-human-ec"].to_numpy(), [2.5, 97.5]
    )
    print(f"95% interval for {m2}: [{lower:.3f}, {upper:.3f}]")

    jdf = pd.merge(
        left=mean_df[mean_df["model"] == m1],
        right=mean_df[mean_df["model"] == m2],
        on=["bootstrap_id"],
    )
    jdf["ec-delta"] = jdf["model-human-ec_x"] - jdf["model-human-ec_y"]

    lower, upper = np.percentile(jdf["ec-delta"].to_numpy(), [2.5, 97.5])
    print(
        f"mean delta: {np.mean(jdf['ec-delta'].to_numpy()):.3f} 95% interval for delta: [{lower:.3f}, {upper:.3f}]"
    )

    t_statistic, p_value = stats.ttest_1samp(jdf["ec-delta"].to_numpy(), popmean=0)

    if plot:
        print(f"t = {t_statistic:.3f}, p = {p_value}")
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        sns.histplot(data=jdf, x="ec-delta", ax=ax)
        sns.despine()

        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        ax.grid(axis="both")
        sns.regplot(data=jdf, x="model-human-ec_x", y="model-human-ec_y", ax=ax)
        mini = np.min([jdf["model-human-ec_x"].min(), jdf["model-human-ec_x"].min()])
        maxi = np.max([jdf["model-human-ec_x"].max(), jdf["model-human-ec_x"].max()])
        ax.plot(
            np.linspace(mini, maxi, 100), np.linspace(mini, maxi, 100), linestyle=":"
        )
        sns.despine()
    return p_value

In [ ]:
m0 = "clip"
m1 = "BiTM_resnetv2_101x1"
m2 = "BiTM_resnetv2_152x2"
m3 = "BiTM_resnetv2_50x1"
m4 = "resnet50_l2_eps5"
calc_p_value(df, m0, m1)

In [ ]:
calc_p_value(df, m1, m2)

In [ ]:
calc_p_value(df, m2, m3)

In [ ]:
# Calculating all p-values between adjacent models, using the broken method
for idx, m1 in enumerate(models[:-2]):
    m2 = models[idx + 1]
    p = calc_p_value(df, m1, m2, False)
    print(m1, m2, p)

## Correct Approach

Logic is: Represent the null hypothesis of models having an equal EC by, for every human-model1-model2 triplet, randomly flipping (p=0.5) the model responses.

In [ ]:
import subprocess

script_path = "bootstrap_rebuttal.py"

for idx, m1 in enumerate(models[:-2]):
    m2 = models[idx + 1]

    command = ["python", script_path, "-m1", m1, "-m2", m2]
    result = subprocess.run(command, capture_output=True, text=True)

    print(f"Command: {' '.join(command)}")
    print(f"Return code: {result.returncode}")
    print(f"Output:\n{result.stdout}")
    if result.stderr:
        print(f"Errors:\n{result.stderr}")

In [ ]:
def calc_p_value_correctly(model1, model2, plot=False):
    print(f"Comparing {model1} and {model2}")

    null_hypo_df = pd.read_parquet(
        f"data/rebuttal_model_wise_bootstrapped_ecs_standard_1000_{model1}_{model2}.parquet",
        engine="pyarrow",
    )
    null_hypo_df["model"] = null_hypo_df["model"].astype(str)

    # take the mean over conditions
    null_hypo_exp_mean_df = null_hypo_df.groupby(
        ["bootstrap_id", "experiment", "model"], observed=True, as_index=False
    ).mean(numeric_only=True)

    # take the average across the experiments
    null_hypo_mean_df = null_hypo_exp_mean_df.groupby(
        ["bootstrap_id", "model"], observed=True, as_index=False
    ).mean(numeric_only=True)

    null_hypo_joined_df = pd.merge(
        left=null_hypo_mean_df[null_hypo_mean_df["model"] == model1],
        right=null_hypo_mean_df[null_hypo_mean_df["model"] == model2],
        on=["bootstrap_id"],
    )
    null_hypo_joined_df["ec-delta"] = (
        null_hypo_joined_df["model-human-ec_x"]
        - null_hypo_joined_df["model-human-ec_y"]
    )

    lower, upper = np.percentile(
        null_hypo_joined_df["ec-delta"].to_numpy(), [2.5, 97.5]
    )
    print(
        f"mean delta under null: {np.mean(null_hypo_joined_df['ec-delta'].to_numpy()):.3f} 95% interval for delta: [{lower:.3f}, {upper:.3f}]"
    )

    real_delta = null_hypo_joined_df[null_hypo_joined_df["bootstrap_id"] == 0][
        "ec-delta"
    ].values[0]
    print("real EC delta: ", real_delta)

    all_deltas = np.abs(
        null_hypo_joined_df[null_hypo_joined_df["bootstrap_id"] > 0]["ec-delta"].values
    )
    larger_deltas = all_deltas > np.abs(real_delta)
    proportion = np.mean(larger_deltas)
    print(f"Proportion of values under null more extreme than real_delta: {proportion}")

    if plot:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        sns.histplot(data=null_hypo_joined_df, x="ec-delta", ax=ax)
        sns.despine()

    return proportion

In [ ]:
model1s = []
model2s = []
props = []

for idx, model1 in enumerate(models[:-2]):
    model2 = models[idx + 1]

    prop = calc_p_value_correctly(model1, model2)

    model1s.append(model1)
    model2s.append(model2)
    props.append(prop)

res_df = pd.DataFrame({"Model 1": model1s, "Model 2": model2s, "P Value": props})

In [ ]:
display(res_df)

In [ ]:
# Getting the data for comparing only the second-best model to all others, sequentially

import subprocess

script_path = "bootstrap_rebuttal.py"

m1 = models[1]
for m2 in models[2:]:

    command = ["python", script_path, "-m1", m1, "-m2", m2]
    result = subprocess.run(command, capture_output=True, text=True)

    print(f"Command: {' '.join(command)}")
    print(f"Return code: {result.returncode}")
    print(f"Output:\n{result.stdout}")
    if result.stderr:
        print(f"Errors:\n{result.stderr}")

In [ ]:
model1s = []
model2s = []
props = []

model1 = models[1]
for model2 in models[2:]:

    prop = calc_p_value_correctly(model1, model2)

    model1s.append(model1)
    model2s.append(model2)
    props.append(prop)

other_res_df = pd.DataFrame({"Model 1": model1s, "Model 2": model2s, "P Value": props})

In [ ]:
display(other_res_df)